<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span><ul class="toc-item"><li><span><a href="#Модель-LogReg" data-toc-modified-id="Модель-LogReg-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Модель LogReg</a></span></li><li><span><a href="#LinearSVC" data-toc-modified-id="LinearSVC-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>LinearSVC</a></span></li></ul></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Общий-вывод" data-toc-modified-id="Общий-вывод-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Общий вывод</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Проект для «Викишоп» без BERT

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

ЦЕЛЬ
- классифицировать комментарии на позитивные и негативные

ЗАДАЧА
- обучить модель

In [1]:
# !pip install spacy
# !python -m spacy download en_core_web_sm

In [2]:
# подготавливаем рабочее пространство,
# загружаем библиотеки
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time

import nltk
from nltk.corpus import stopwords as nltk_stopwords
nltk.download('stopwords')


from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

import nltk
from nltk.corpus import stopwords as nltk_stopwords
from sklearn.feature_extraction.text import TfidfVectorizer

pd.set_option('display.max_columns', None)

import re
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score

from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')

from sklearn.feature_extraction.text import CountVectorizer
from sklearn import metrics

from sklearn.svm import LinearSVC
from sklearn.model_selection import GridSearchCV, train_test_split, cross_val_score
from sklearn.metrics import f1_score

from sklearn.model_selection import GridSearchCV, cross_val_score
import numpy as np
import time

from nltk.tokenize import word_tokenize 
from nltk.stem import WordNetLemmatizer 
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords 
from nltk.corpus import wordnet


nltk.download('punkt')  # Токенизатор
nltk.download('averaged_perceptron_tagger')  # POS-теггер
nltk.download('wordnet')  # WordNet для лемматизации


[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /home/jovyan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [3]:
data = pd.read_csv('/datasets/toxic_comments.csv')
display(data.info())
display(data.head(5))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB


None

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0


Текст нуждается в очистке >>>

## Подготовка

In [4]:
stopwords = set(stopwords.words('english'))

In [5]:
lemmatizer = WordNetLemmatizer()

In [6]:
def get_wordnet_pos(treebank_tag):
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN  

def clear_and_lemmatize(text):
    text = re.sub(r'[^a-zA-Z ]', ' ', text)
    text = text.lower()
    
    token = nltk.word_tokenize(text)
    pos_tags = nltk.pos_tag(token)
    
    text = [word for word, tag in pos_tags if word not in stopwords]
    text = [lemmatizer.lemmatize(word, get_wordnet_pos(tag)) for word, tag in pos_tags if word not in stopwords]
    text = ' '.join(text)
    
    return text

<span style="color: blue; font-weight: bold">Комментарий ревьюера 3: </span> ⚠️\
<span style="color: darkorange"> Условие по `stopwords` выполняется два раза для каждого примера.</span>

In [7]:
data['text'] = data['text'].apply(clear_and_lemmatize)
data.head()

,Unnamed: 0,text,toxic
0,0,explanation edits make username hardcore metal...,0
1,1,aww match background colour seemingly stick th...,0
2,2,hey man really try edit war guy constantly rem...,0
3,3,make real suggestion improvement wonder sectio...,0
4,4,sir hero chance remember page,0


In [8]:
# data['text'] = data['text'].apply(clear_text)
# corpus = data['text'].values.astype('U')
# corpus[0] = lemmatize(corpus[0])

Получили чистый и лемматизированный текст )

## Обучение

In [9]:
X = data['text']
y = data['toxic'].values
X_train, X_test, y_train, y_test =  train_test_split(X, y, shuffle=False, test_size=0.25, random_state = 42)

<font color='blue'><b>Комментарий ревьюера: </b></font> ✔️\
<font color='green'> Здорово , что у нас есть выборка для тестов!</font>

In [10]:
# считаем величину TF-IDF для обучающей выборки
count_tf_idf_train = TfidfVectorizer(stop_words=stopwords)
tf_idf_train = count_tf_idf_train.fit_transform(X_train)
display(tf_idf_train.shape)

(119469, 131288)

In [11]:
# считаем величину TF-IDF для тестовой выборки
count_tf_idf_test = TfidfVectorizer(stop_words=stopwords)
tf_idf_test = count_tf_idf_train.transform(X_test)
display(tf_idf_test.shape)

(39823, 131288)

### Модель LogReg

In [12]:
param_grid = {
    'solver': ['liblinear', 'saga'],
    'penalty' : ['l1', 'l2']
}

model =  LogisticRegression(random_state=1, max_iter=200)
gs = GridSearchCV(estimator=model,
                  param_grid=param_grid, 
                  scoring='f1', 
                  cv=3, 
                  verbose=3)

# Обучаем с помощью кросс-валидации
gs.fit(tf_idf_train, y_train)

# Получаем лучшие гиперпараметры
best_params_lr = gs.best_params_

# Создаем новую модель с лучшими гиперпараметрами
best_model_lr = LogisticRegression(**best_params_lr)

start_fit_time = time.time()

# Обучаем модель на всем обучающем наборе данных
best_model_lr.fit(tf_idf_train, y_train)

fit_time_lr = time.time() - start_fit_time

Fitting 3 folds for each of 4 candidates, totalling 12 fits
[CV 1/3] END ...................penalty=l1, solver=liblinear; total time=   1.0s
[CV 2/3] END ...................penalty=l1, solver=liblinear; total time=   1.0s
[CV 3/3] END ...................penalty=l1, solver=liblinear; total time=   0.9s
[CV 1/3] END ........................penalty=l1, solver=saga; total time=  18.3s
[CV 2/3] END ........................penalty=l1, solver=saga; total time=  19.4s
[CV 3/3] END ........................penalty=l1, solver=saga; total time=  20.3s
[CV 1/3] END ...................penalty=l2, solver=liblinear; total time=   8.2s
[CV 2/3] END ...................penalty=l2, solver=liblinear; total time=   7.3s
[CV 3/3] END ...................penalty=l2, solver=liblinear; total time=   7.7s
[CV 1/3] END ........................penalty=l2, solver=saga; total time=   1.5s
[CV 2/3] END ........................penalty=l2, solver=saga; total time=   1.8s
[CV 3/3] END ........................penalty=l2, 

In [13]:
start_predict_time = time.time()
predictions_lr = best_model_lr.predict(tf_idf_train)
predict_time_lr = time.time() - start_predict_time

In [14]:
# Оценка F1 с кросс-валидацией
cv_f1_lr = cross_val_score(best_model_lr, tf_idf_train, y_train, scoring='f1', cv=3)
f1_lr = cv_f1_lr.mean()

print("Лучшие гиперпараметры модели LogisticRegression: ", best_params_lr)
print("F1 LogisticRegression (кросс-валидация): %.2f" % f1_lr)

Лучшие гиперпараметры модели LogisticRegression:  {'penalty': 'l1', 'solver': 'liblinear'}
F1 LogisticRegression (кросс-валидация): 0.76


In [15]:
# # Оценка F1 LogReg на тестовой выборке
# f1_test = f1_score(y_test, best_model_lr.predict(tf_idf_test))
# print("F1 LogisticRegression  на тестовой выборке: %.2f" % f1_test)

### LinearSVC

In [16]:
from sklearn.svm import LinearSVC
from sklearn.model_selection import GridSearchCV, cross_val_score
import time

# Задаем параметры для поиска по сетке
param_grid = {
    'C': [0.1, 1, 10],            
    'max_iter': [1000, 2000, 5000] 
}

# Инициализация модели LinearSVC
model = LinearSVC(random_state=12345)

# Инициализация GridSearchCV
gs = GridSearchCV(model, param_grid=param_grid, scoring='f1', cv=3, verbose=1)

# Обучаем с помощью кросс-валидации
gs.fit(tf_idf_train, y_train)

# Получаем лучшие гиперпараметры
best_params_svc = gs.best_params_

# Создаем новую модель с лучшими гиперпараметрами
best_model_svc = LinearSVC(**best_params_svc, random_state=12345)

# Обучаем модель на всем обучающем наборе данных
start_fit_time = time.time()
best_model_svc.fit(tf_idf_train, y_train)
fit_time_svc = time.time() - start_fit_time

# Прогнозируем
start_predict_time = time.time()
predictions_svc = best_model_svc.predict(tf_idf_train)
predict_time_svc = time.time() - start_predict_time

# Оценка F1 с кросс-валидацией
cv_f1_svc = cross_val_score(best_model_svc, tf_idf_train, y_train, scoring='f1', cv=3)
f1_svc = cv_f1_svc.mean()

# Вывод результатов
print("Лучшие гиперпараметры модели LinearSVC: ", best_params_svc)
print("F1 LinearSVC (кросс-валидация): %.2f" % f1_svc)

Fitting 3 folds for each of 9 candidates, totalling 27 fits
Лучшие гиперпараметры модели LinearSVC:  {'C': 1, 'max_iter': 1000}
F1 LinearSVC (кросс-валидация): 0.76


<span style="color: blue; font-weight: bold">Комментарий ревьюера : </span> ✔️\
<span style="color: green"> 👍</span>

## Выводы

In [17]:
results = pd.DataFrame({'Model': ['LogisticRegression', 'LinearSVC'], 'Время обучения(с)': [fit_time_lr, fit_time_svc], 'Время предсказания (с)': [predict_time_lr, predict_time_svc], 'f1 (кросс-валидация)': [f1_lr, f1_svc]})
display(results)

,Model,Время обучения(с),Время предсказания (с),f1 (кросс-валидация)
0,LogisticRegression,1.457330,0.014356,0.756772
1,LinearSVC,1.091174,0.015041,0.763344


<font color='blue'><b>Комментарий ревьюера: </b></font> ✔️\
<font color='green'> Отлично, что есть табличка результатов для сравнения.</font>

Обе модели неплохо показали себя на кросс валидации, теперь однако модель SVC показала себя немного лучше поэтому именно ее мы будем проверять на тестовой выборке)

In [18]:
# Оценка F1 LinearSVC на тестовой выборке
f1_test = f1_score(y_test, best_model_svc.predict(tf_idf_test))
print("F1 LinearSVC  на тестовой выборке: %.2f" % f1_test)

F1 LinearSVC  на тестовой выборке: 0.77


<font color='blue'><b>Комментарий ревьюера: </b></font> ✔️\
<font color='green'>Тестирование проведено верно.</font>

## Общий вывод

В ходе данной работы была получена модель способная классифицировать комментарии на позитивные и негативные. Для этого были проделаны  следующие шаги:

1) Выгрузка и подготовка данных

2) Обучены две разные модели с подбором гиперпараметров LogReg и LinearSVC. Сравнили время обучени и предсказания моделей, и их f1.

3) Определили, что Лучшей оказалась модель LinearSVC:  {'C': 1, 'max_iter': 1000} с F1 (кросс-валидация): 0.76. Также, данная модель хорошо повела себя на тестовой выборке и F1 на тестовой выборке составила 0.77 и что данную модель можно совершенно точно без опасений рекоммендовать клиентам)

## Чек-лист проверки

- [x]  Jupyter Notebook открыт
- [x]  Весь код выполняется без ошибок
- [x]  Ячейки с кодом расположены в порядке исполнения
- [x]  Данные загружены и подготовлены
- [x]  Модели обучены
- [x]  Значение метрики *F1* не меньше 0.75
- [x]  Выводы написаны